In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install underthesea
!pip install fairseq
!pip install fastBPE
!pip install tensorflow
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 62.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291640 sha256=caa28af09572e92b218a

In [3]:
import bs4
import time
import re
import requests
import pickle
# from underthesea import word_tokenize
from transformers import RobertaForSequenceClassification, RobertaConfig
import torch
import argparse
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
import json
from tqdm import tqdm

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')


Using device "cuda"


In [5]:
model = 'base'
data_path = '/content/drive/MyDrive/pRoBERTa/source/result/result_private-test_rationale_VNsbert_no_thredhold.json' # Đường dẫn data muốn label
path_label_bin = '/content/drive/MyDrive/FEVER/output_model_verify/pytorch_model.bin' # Đường dẫn tới file bin model trained
save_path = "/content/drive/MyDrive/pRoBERTa/source/result" # Đường dẫn lưu file sau khi label
name_file_labeled = "private-test_labels_no_thredhold_2.json"

# Encoder

In [ ]:
model = 'base'
class Encoder:
  def __init__(self):
    # Load BPE encoder
    parser = argparse.ArgumentParser()
    parser.add_argument(
      '--bpe-codes',
      default=f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/bpe.codes',
      required=False,
      type=str,
      help='path to fastBPE BPE'
    )

    args, unknown = parser.parse_known_args()
    self.bpe = fastBPE(args)

    # Load the dictionary
    self.vocab = Dictionary()
    self.vocab.add_from_file(f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/dict.txt')

  def __call__(self, sent1, sent2):
    subwords = '[CLS]' + self.bpe.encode(sent1) + '[SEP]' + self.bpe.encode(sent2) + '[SEP]'
    encoded_sent = self.vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
    encoded_sent = [encoded_sent]
    encoded_sent = pad_sequences(encoded_sent, maxlen=256, dtype='long', value=1, truncating='post', padding='post')
    mask = [int(token_id != 1) for token_id in encoded_sent[0]]
    mask = [mask]

    return torch.tensor(encoded_sent), torch.tensor(mask)

# Load trained model

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

NUM_CLASSES = 2

# load file config
config = RobertaConfig.from_pretrained(
    f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/model.bin',
    config=config
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/pRoBERTa/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaModel

class CustomRobertaClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super(CustomRobertaClassificationHead, self).__init__()
        self.dense = nn.Linear(768, 768)
        self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=256, batch_first=True)
        self.fc1 = nn.Linear(256, 128)
        self.out_proj = nn.Linear(128, num_labels)

    def forward(self, hidden_states):
        x = self.dense(hidden_states)  # Ánh xạ hidden_states qua lớp tuyến tính dense
        x = torch.relu(x)  # Áp dụng hàm kích hoạt ReLU
        x = self.dropout(x)  # Áp dụng dropout
        _, (h_n, _) = self.lstm(x)  # Đưa x qua LSTM và lấy hidden state cuối cùng
        x = self.fc1(h_n[-1])
        x = self.out_proj(x)

        return x

class CustomRobertaForSequenceClassification_Label(nn.Module):
    def __init__(self, model, num_labels):
        super(CustomRobertaForSequenceClassification_Label, self).__init__()

        self.roberta = model
        self.roberta.classifier = CustomRobertaClassificationHead(hidden_size=768, num_labels=num_labels)

    def forward(self, input_ids, token_type_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        return outputs

    def save_pretrained(self, PATH):
       # Create the directory if it doesn't exist
      os.makedirs(PATH, exist_ok=True)

      # Save the model's state_dict to a file
      model_path = os.path.join(PATH, 'model.bin')
      torch.save(self.state_dict(), model_path)

In [ ]:
label_model = CustomRobertaForSequenceClassification_Label(BERT_SA, NUM_CLASSES)
label_model.load_state_dict(torch.load(path_label_bin))

<All keys matched successfully>

In [ ]:
label = {
    0: "REFUTED",
    1: "SUPPORTED"
}

In [ ]:
label_model.cuda()

CustomRobertaForSequenceClassification_Label(
  (roberta): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(64001, 768, padding_idx=0)
        (position_embeddings): Embedding(258, 768, padding_idx=0)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelf

In [ ]:
encoder = Encoder()

In [ ]:
claim = "Hiệu trưởng nơi đăng ký thi sẽ không nhận hồ sơ của thí sinh sau buổi thi cuối một tuần"
sent = "Về thủ tục, thí sinh nộp hồ sơ cho hiệu trưởng nơi đăng ký dự thi chậm nhất một tuần sau buổi thi cuối cùng của kỳ thi."
token_ids, mask = encoder(claim, sent)
token_ids = token_ids.to(device)
mask = mask.to(device)
print(token_ids)
print(mask)
outputs = label_model(
  token_ids,
  token_type_ids=None,
  attention_mask=mask
)

logits = outputs[0]
logits = logits.detach().cpu().numpy()


print('Label index: ', label[np.argmax(logits)])

tensor([[    3,   676,   189,  1813,   765,   201,    38,    17,   154,  1005,
          6829,     7, 32000,   418,    53,   391,   201,   352,    16,     3,
          2183, 55582,  1395,     4, 32000,   418,  1140,  1005,  6829,    13,
          3256,   676,   189,  1813,   765,   930,   201,  1220,    67,    16,
           386,    53,   391,   201,   352,    81,     7,   560,  7644,     3,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

# Chạy trên data

In [ ]:
label_model.cuda()

CustomRobertaForSequenceClassification_Label(
  (roberta): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(64001, 768, padding_idx=0)
        (position_embeddings): Embedding(258, 768, padding_idx=0)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelf

In [ ]:
def logits_scale(logits):
    # Khoảng mới
    new_min = -1
    new_max = 1

    # Tính toán phép tỷ lệ tuyến tính
    scaled_x = [(new_max - new_min) * (value - (-5)) / ((5) - (-5)) + new_min for value in logits]
    return scaled_x

In [ ]:
import json

# Đọc dữ liệu từ file JSON
with open(data_path) as file:
    converted_data = json.load(file)

# Lặp qua mỗi ID trong data
from zmq.constants import NULL
encoder = Encoder()

for id, item in tqdm(converted_data.items()):
  labels_encoder = {
      1: "REFUTED",
      0: "SUPPORTED"
  }
  claim = item['claim']
  try:
    evidence = item['evidence_predict']
  except:
    evidence = None
  if evidence == None:
    label_predict = "NEI"
  else:
    token_ids, mask = encoder(claim, evidence)
    token_ids = token_ids.to(device)
    mask = mask.to(device)
    with torch.no_grad():
      outputs = label_model(
        token_ids,
        token_type_ids=None,
        attention_mask=mask
      )

      logits = outputs[0]
      logits = logits.detach().cpu().numpy()

      lo_scale = logits_scale(logits[0])
      # Áp dụng hàm softmax
      probabilities = np.exp(lo_scale) / np.sum(np.exp(lo_scale))

      # Tính phần trăm khả năng cho mỗi lớp
      percentages = probabilities * 100

      logits = np.argmax(logits)
      label_predict = labels_encoder[logits]

  print(label_predict)

  converted_data[id]['verdict_predict'] = label_predict
  converted_data[id]['evidence_predict'] = evidence
  # converted_data[id]['percentage_labels'] = percentages[logits]

  0%|          | 3/5396 [00:00<07:47, 11.54it/s]

SUPPORTED
SUPPORTED
SUPPORTED


  0%|          | 7/5396 [00:00<06:04, 14.80it/s]

SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED


  0%|          | 9/5396 [00:00<06:06, 14.69it/s]

...

SUPPORTED
SUPPORTED
SUPPORTED
REFUTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED


100%|█████████▉| 5393/5396 [02:03<00:00, 42.70it/s]

SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED
SUPPORTED


100%|██████████| 5396/5396 [02:03<00:00, 43.63it/s]

SUPPORTED
SUPPORTED


# Lưu kết quả

In [ ]:
import os
import json

def save_result(PATH, result_dict):
    # Tạo thư mục nếu nó chưa tồn tại
    os.makedirs(PATH, exist_ok=True)
    # Lưu từ điển vào tệp JSON
    result_path = os.path.join(PATH, name_file_labeled)

    with open(result_path, "w", encoding="utf-8") as file:
        json.dump(converted_data, file, ensure_ascii=False)

In [ ]:
save_result(save_path, converted_data)

In [ ]:
import json
with open("/content/drive/MyDrive/pRoBERTa/data/converted_data_private-test_split.json") as file:
    public_test_split = json.load(file)
with open("/content/drive/MyDrive/pRoBERTa/data/converted_data_private-test_split.json") as file:
    public_test_rationale = json.load(file)
public_result = {}
for id, item in converted_data.items():
  public_result[id] = {}
  public_result[id]['verdict'] = item["verdict_predict"]
  public_result[id]['evidence'] = public_test_split[id]["context"][item['idx_evidence']] if item["evidence_predict"] != None else ""
  # public_result[id]['evidence'] = item['evidence_predict'].replace("_", " ") if item["evidence_predict"] != None else ""
  public_result[id]['percentages_rationale'] = item['percentages_rationale']


with open("/content/drive/MyDrive/pRoBERTa/source/result/private_result_no_thredhold_2.json", "w", encoding="utf-8") as file:
  json.dump(public_result, file, ensure_ascii=False)